In [1]:
# Code file for pre-processing data: cropping, padding, creating label mask, and saving as tif file in data folder
# Does not save any ignition point or time information (next step)

import numpy as np
import pandas as pd
import os
import rasterio
import geopandas as gpd
from rasterio.warp import reproject, Resampling
from rasterio.windows import Window

# Load metadata and filter for Maule region with fire scar
df = pd.read_excel("FireScar_CL_Summary_1985-2018.xlsx")
df = df[(df['FireScar'] == 1) & (df['Region_CONAF'] == 'Maule')].reset_index(drop=True)

# Output directory
output_dir = "data"
os.makedirs(output_dir, exist_ok=True)

def process_fire_event(fire_id, df):
    example_fire = df.loc[fire_id]
    fire_season = example_fire["FireSeason"]
    folder = f"Fire_CL-ML_Maule/FireScar_CL-ML_Maule_{fire_season}"
    
    # Paths to raster and vector files
    pre_fire_path = os.path.join(folder, example_fire["PreFireImgName"])
    post_fire_path = os.path.join(folder, example_fire["PostFireImgName"])
    fire_scar_vector_path = os.path.join(folder, example_fire["FireScarVectorName"])
    fire_scar_path = os.path.join(folder, example_fire["FireScarImgName"])
    severity_path = os.path.join(folder, example_fire["SeverityImgName"])

    merged_filename = os.path.join(output_dir, f"merged_{fire_id}.tif")
    mask_filename = os.path.join(output_dir, f"mask_{fire_id}.tif")

    # Load ignition point
    fire_scar_vector = gpd.read_file(fire_scar_vector_path).to_crs(epsg=4326)
    target_lat = fire_scar_vector['IgnLat'][0]
    target_lon = fire_scar_vector['IgnLon'][0]

    # Crop and save merged post-fire image
    with rasterio.open(post_fire_path) as src:
        xsize, ysize = 512, 512
        target_x, target_y = src.index(target_lon, target_lat)
        xoff = target_x - xsize // 2
        yoff = target_y - ysize // 2
        window = Window(xoff, yoff, xsize, ysize).round_offsets()
        transform = src.window_transform(window)
        profile = src.profile
        profile.update({
            'height': ysize,
            'width': xsize,
            'transform': transform
        })
        data = src.read(window=window, boundless=True, fill_value=np.nan)
        with rasterio.open(merged_filename, 'w', **profile) as dst:
            dst.write(data)

    # Check dimensions
    with rasterio.open(merged_filename) as check:
        if check.width != 512 or check.height != 512:
            print(f"⚠️ Dimension error for fire {fire_id}")

    # Create fire scar label mask
    with rasterio.open(merged_filename) as cropped_src:
        dst_crs = cropped_src.crs
        dst_transform = cropped_src.transform
        dst_shape = cropped_src.shape
        dst_meta = cropped_src.meta.copy()

    with rasterio.open(fire_scar_path) as fire_src:
        fire_scar_data = fire_src.read(1)
        src_crs = fire_src.crs
        src_transform = fire_src.transform

    resampled_fire_scar = np.zeros(dst_shape, dtype=fire_scar_data.dtype)
    reproject(
        source=fire_scar_data,
        destination=resampled_fire_scar,
        src_transform=src_transform,
        src_crs=src_crs,
        dst_transform=dst_transform,
        dst_crs=dst_crs,
        resampling=Resampling.nearest
    )

    dst_meta.update({
        'height': dst_shape[0],
        'width': dst_shape[1],
        'transform': dst_transform,
        'crs': dst_crs,
        'count': 1,
        'dtype': fire_scar_data.dtype
    })

    with rasterio.open(mask_filename, 'w', **dst_meta) as dst:
        dst.write(resampled_fire_scar, 1)

    print(f"✅ Fire {fire_id} processed and saved to '{output_dir}'")

# Run the loop over all events
for fire_id in range(len(df)):
    try:
        process_fire_event(fire_id, df)
    except Exception as e:
        print(f"❌ Error processing fire {fire_id}: {e}")


✅ Fire 0 processed and saved to 'data'
✅ Fire 1 processed and saved to 'data'
✅ Fire 2 processed and saved to 'data'
✅ Fire 3 processed and saved to 'data'
✅ Fire 4 processed and saved to 'data'
✅ Fire 5 processed and saved to 'data'
✅ Fire 6 processed and saved to 'data'
✅ Fire 7 processed and saved to 'data'
✅ Fire 8 processed and saved to 'data'
✅ Fire 9 processed and saved to 'data'
✅ Fire 10 processed and saved to 'data'
✅ Fire 11 processed and saved to 'data'
✅ Fire 12 processed and saved to 'data'
✅ Fire 13 processed and saved to 'data'
✅ Fire 14 processed and saved to 'data'
✅ Fire 15 processed and saved to 'data'
✅ Fire 16 processed and saved to 'data'
✅ Fire 17 processed and saved to 'data'
✅ Fire 18 processed and saved to 'data'
✅ Fire 19 processed and saved to 'data'
✅ Fire 20 processed and saved to 'data'
✅ Fire 21 processed and saved to 'data'
✅ Fire 22 processed and saved to 'data'
✅ Fire 23 processed and saved to 'data'
✅ Fire 24 processed and saved to 'data'
✅ Fire 25 